# PROJECT TITLE: FINDING THE MOST INTERESTING TWEETS AND FAN SENTIMENT DURING A GAME TELECAST IN REALTIME

# PART 6: SET TEAM / SENTIMENT TWEET REFERS TO:

In [1]:
!pip install vaderSentiment

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1076)'))) - skipping


In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
dfEv = pd.read_csv('events_proc.csv')
dfEv.drop(['Unnamed: 0'], axis = 1, inplace = True)
dfEv

,Timestamp,ExtraMins,Team1,Team2,Player,Team,Event,MatchNo,EventID,EventHr,EventMnt,Time,Date
0,0,0,FRANCE,ARGENTINA,NaN,NaN,START,1,1101,17,0,17:00,30-06-2018
1,1,0,FRANCE,ARGENTINA,GIROUD,FRANCE,FOUL,1,1102,17,1,17:01,30-06-2018
2,3,0,FRANCE,ARGENTINA,POGBA,FRANCE,FOUL,1,1103,17,3,17:03,30-06-2018
3,3,0,FRANCE,ARGENTINA,MATUIDI,FRANCE,FOUL,1,1104,17,3,17:03,30-06-2018
4,8,0,FRANCE,ARGENTINA,MASCHERANO,ARGENTINA,FOUL,1,1105,17,8,17:08,30-06-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,90,1,FRANCE,CROATIA,VRSALJKO,CROATIA,FOUL,16,16230,19,34,19:34,15-07-2018
1462,90,2,FRANCE,CROATIA,VRSALJKO,CROATIA,FOUL,16,16231,19,35,19:35,15-07-2018
1463,90,3,FRANCE,CROATIA,VRSALJKO,CROATIA,YELLOW CARD,16,16232,19,36,19:36,15-07-2018
1464,90,4,FRANCE,CROATIA,FEKIR,FRANCE,FOUL,16,16233,19,37,19:37,15-07-2018


In [4]:
def set_team_from_tweet(fn):
    ################################################################
    # STEP 1: INITIALIZE ARRAY'S TO COLLECT EVENT DATA INSIGHTS:
    # A. PLAYER TEAM SET - SET OF ALL PLAYER AND TEAMS THEY BELONG TO
    # B. ABB SET - SET OF ALL ABB'S, TWEET CONTENT INCLUDES ABB'S
    # C. KEY WORD SET - SET OF PLAYER NAMES, TEAM NAMES, ABB'S, COMBINED ABB'S
    # D. TEAM ABB SET - RELATES ABB TO TEAM NAME
    # E. TEAM-TEAM ABB SET - REALTES COMBINED ABB TO BOTH TEAM NAMES
    # F. TEAM LIST

    # Key Word Array
    kw_arr = []
    # Keyword Set
    kw_set = []

    # Player Team Array
    pt_arr = []
    # Player Team Set
    pt_set = []

    # Abbreviation Array
    abb_arr = []
    # Abbreviation Set
    abb_set = []

    # Team Abbreviation Array
    ta_arr = []
    # Team Abbreviation Set
    ta_set = []

    # Team-Team Abbreviation Array
    tta_arr = []
    # Team-Team Abbreviation Set
    tta_set = []

    # Team List
    team_list = []
    # Team Set
    team_set = []

    #################################################################################
    # STEP 2: BUILD THE RESPECTIVE SETS

    dfEv.Player = dfEv.Player.fillna('')
    dfEv.Team = dfEv.Team.fillna('')

    for i in range(1466):
        if dfEv['Player'][i] != '':
            pt_arr.append([dfEv['Player'][i].lower(), dfEv['Team'][i].lower()])
            ta_arr.append([dfEv['Team'][i].lower(), dfEv['Team'][i][:3].lower()])
            team_list.append(dfEv['Team'][i].lower())
            kw_arr.append(dfEv['Player'][i].lower())
            kw_arr.append(dfEv['Team'][i].lower())

    for i in range(1466):
        a = dfEv['Team1'][i][:3].lower()
        b = dfEv['Team2'][i][:3].lower()
        c = a + b
        abb_arr.append([a, b, c])
        tta_arr.append([dfEv['Team1'][i].lower(), dfEv['Team2'][i].lower(), c])
        kw_arr.append(a)
        kw_arr.append(b)
        kw_arr.append(c)


    for i in pt_arr:
        if i not in pt_set:
            pt_set.append(i)

    for i in abb_arr:
        if i not in abb_set:
            abb_set.append(i)

    for i in kw_arr:
        if i not in kw_set:
            kw_set.append(i)

    for i in ta_arr:
        if i not in ta_set:
            ta_set.append(i)

    for i in tta_arr:
        if i not in tta_set:
            tta_set.append(i)

    for i in team_list:
        if i not in team_set:
            team_set.append(i)

    ################################################################################
    # STEP 3: COLLECT BAG OF WORDS FROM TWEET DATA, HASHTAG DATA, USER MENTIONED NAMES / IDS

    df = pd.read_csv(fn)
    df.drop(['Unnamed: 0', 'HTs', 'UMN', 'UMID'], axis = 1, inplace = True)

    df1 = df["Tweet_Extra"]
    # Tweet Exra Content Array - Contains HTs, UMN, UMID data
    te = []
    for i in df1:
        x = i.split('[')
        y = x[1].split(']')
        z = y[0].split("'")
        te.append(z)
    for i in te:
        i.remove('')
    for i in te:
        for j in i:
            if any(c.isalpha() for c in j):
                pass
            else:
                i.remove(j)

    df1 = df["Tweet_Proc"]
    # Tweet Content Array - Contains Tweet Data
    tc = []
    for i in df1:
        x = i.split('[')
        y = x[1].split(']')
        z = y[0].split("'")
        tc.append(z)
    for i in tc:
        i.remove('')
    for i in tc:
        for j in i:
            if any(c.isalpha() for c in j):
                pass
            else:
                i.remove(j)

    ############################################################################################################
    # STEP 4: FIND INTERSECTIONS OF KEYWORDS AND TWEETS TO DRAW INSIGHTS AS TO WHICH TEAM IS BEING REFERRED TO:

    # ARR1 - INTERSECTION OF KEYWORDS AND TWEET CONTENT
    arr1 = [] 
    set_A = set(kw_set)
    for i in range(len(tc)):
        set_B = set(tc[i])
        arr1.append(list(set_A.intersection(set_B)))

    # ARR2 - INTERSECTION OF KEYWORDS AND TWEET EXTRAS
    arr2 = []
    for i in range(len(te)):
        set_C = set(te[i])
        arr2.append(list(set_A.intersection(set_C)))

    ##############################################################################################
    # STEP 5: IF PLAYER NAME / ABB KEYWORDS ARE FOUND ADD TEAM NAME ALSO
    for i in arr1:
        for j in i:
            for k in pt_set:
                if k[0] == j:
                    i.append(k[1])

    for i in arr2:
        for j in i:
            for k in pt_set:
                if k[0] == j:
                    i.append(k[1])

    for i in arr1:
        for j in i:
            for k in ta_set:
                if k[1] == j:
                    i.append(k[0])

    for i in arr2:
        for j in i:
            for k in ta_set:
                if k[1] == j:
                    i.append(k[0])

    for i in arr1:
        for j in i:
            for k in tta_set:
                if k[2] == j:
                    i.append(k[0])
                    i.append(k[1])

    for i in arr2:
        for j in i:
            for k in tta_set:
                if k[2] == j:
                    i.append(k[0])
                    i.append(k[1])

    #############################################################################
    # STEP 6: COMBINE DATA REMOVE KEYWORDS OTHER THAN DERIVED TEAM NAMES

    # ARR3 - COMBINE ARR1 AND ARR2 CONTENT
    arr3 = []
    for i in range(len(arr1)):
        arr3.append(arr1[i] + arr2[i])

    # ARR4 - WILL CONTAIN ONLY TEAM REFERRED TO
    arr4 = []
    for i in arr3:
        tmp = []
        for j in i:
            if j in team_set:
                tmp.append(j)
        arr4.append(tmp)

    df['TeamsRef'] = arr4
    return df

In [5]:
def sentiment_scores(df):
    sid_obj = SentimentIntensityAnalyzer()
    df.Tweet = df.Tweet.fillna('')
    rows = df.shape[0]
    df['PosSent'] = 0
    df['NegSent'] = 0
    df['NeuSent'] = 0

    for i in range(rows):
        sentiment_dict = sid_obj.polarity_scores(df['Tweet'][i])
        df['PosSent'][i] = sentiment_dict['pos']*100
        df['NegSent'][i] = sentiment_dict['neg']*100
        df['NeuSent'][i] = sentiment_dict['neu']*100
            
    return df

In [6]:
df30 = set_team_from_tweet('tweet_3006.csv')
df30 = sentiment_scores(df30)
df30.to_csv('tweet_new_3006.csv')

In [7]:
df01 = set_team_from_tweet('tweet_0107.csv')
df01 = sentiment_scores(df01)
df01.to_csv('tweet_new_0107.csv')

In [8]:
df02 = set_team_from_tweet('tweet_0207.csv')
df02 = sentiment_scores(df02)
df02.to_csv('tweet_new_0207.csv')

In [9]:
df04 = set_team_from_tweet('tweet_0407.csv')
df04 = sentiment_scores(df04)
df04.to_csv('tweet_new_0407.csv')

In [10]:
df10 = set_team_from_tweet('tweet_1007.csv')
df10 = sentiment_scores(df10)
df10.to_csv('tweet_new_1007.csv')

In [11]:
df11 = set_team_from_tweet('tweet_1107.csv')
df11 = sentiment_scores(df11)
df11.to_csv('tweet_new_1107.csv')

In [12]:
df12 = set_team_from_tweet('tweet_1207.csv')
df12 = sentiment_scores(df12)
df12.to_csv('tweet_new_1207.csv')

In [13]:
df15 = set_team_from_tweet('tweet_1507.csv')
df15 = sentiment_scores(df15)
df15.to_csv('tweet_new_1507.csv')

In [14]:
df16 = set_team_from_tweet('tweet_1607.csv')
df16 = sentiment_scores(df16)
df16.to_csv('tweet_new_1607.csv')